In [2]:
import cv2
import mediapipe as mp
import numpy as np
import datetime
import time
import glob
import shutil
#video_path =  'C:/Users/99455/Desktop/DataScience/OEP database/subject1/Yousef1.avi'
path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\Frames2\\*.*"
video_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\testformodel.mp4"

In [3]:
mp_drawing = mp.solutions.drawing_utils

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True,max_num_faces=10,min_detection_confidence=0.5, min_tracking_confidence=0.5)
drawing_spec = mp_drawing.DrawingSpec(thickness=1,circle_radius=1)

In [4]:
#cap = cv2.VideoCapture(0)
#while cap.isOpened():
filename = ""
for file in glob.glob(path):
    check = False
    headPoseDetected = False
    image = cv2.imread(file)
    #image = cap.read()
    #success, image = cap.read()
    start = time.time()
    
    image = cv2.cvtColor(cv2.flip(image,1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    results = face_mesh.process(image)
    image.flags.writeable = True
    
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    
    img_h, img_w, img_c = image.shape
    face_3d = []
    face_2d = []
    filename = file.split('\\')[-1]
        
    if results.multi_face_landmarks:
        if filename == "frame3340.jpg": 
            print("photo")
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                        nose_2d = (lm.x*img_w, lm.y*img_h)  
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)
                        
                    x, y = int(lm.x * img_w), int(lm.y * img_h)
                    face_2d.append([x, y])
                    face_3d.append([x, y, lm.z])
                        
            face_2d = np.array(face_2d,dtype=np.float64)
            
            face_3d = np.array(face_3d,dtype=np.float64)
            
            focal_length = 1 * img_w
            
            cam_matrix = np.array([ [focal_length, 0 ,img_h/2], [0, focal_length, img_w/2], [0,0,1] ])
            
            dist_matrix = np.zeros((4,1),dtype=np.float64)
            
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
            
            rmat, jac = cv2.Rodrigues(rot_vec)
            
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)
            #print(angles)
            
            x = angles[0]*360
            y = angles[1]*360
            z = angles[2]*360
            
            
            #if y < -10 or y > 10 or x < -1 or x > 8:
            old_path =  file
            if x < -10:
                if y > 10:
                    new_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\ExtremeHeadPose\\RightDown\\"+filename
                elif y < -10:
                    new_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\ExtremeHeadPose\\LeftDown\\"+filename
                else:
                    new_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\ExtremeHeadPose\\Down\\"+filename              
                check = True
            elif x > 10:
                if y > 10:
                    new_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\ExtremeHeadPose\\RightUp\\"+filename
                elif y < -10:
                    new_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\ExtremeHeadPose\\LeftUp\\"+filename
                else:
                    new_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\ExtremeHeadPose\\Up\\"+filename              
                check = True
            elif y > 10:
                new_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\ExtremeHeadPose\\Right\\"+filename
                check = True
            elif y < -10:
                new_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\ExtremeHeadPose\\Left\\"+filename
                check = True
#             if x < -10 and  y > -10 and y < 10:
                
#             elif x > 10 and  y > -10 and y < 10:
                
#             elif y < - 15 and x > -10 and x < 10:
                
#             elif y > 10 and x > -10 and x < 10:
                
                
                
            headPoseDetected = True  
            if check == True:
                shutil.move(old_path,new_path)
            #else:
                #text = "Forward"
            
            nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec,trans_vec, cam_matrix, dist_matrix)
            
            p1 =(int(nose_2d[0]), int(nose_2d[1]))
            p2 =(int(nose_2d[0]+y*10), int(nose_2d[1]-x*10))
            
            cv2.line(image,p1,p2, (255,0,0),3)
            
            #cv2.putText(image,text,(20,50),cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0),2)
            cv2.putText(image,"x: "+str(np.round(x,2)), (500,50), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255))
            cv2.putText(image,"y: "+str(np.round(y,2)), (500,100), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255))
            cv2.putText(image,"z: "+str(np.round(z,2)), (500,150), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255))
            cv2.imshow("Head Pose Estimation", image)
            if filename == "frame3340.jpg":
                print("photo")
                cv2.waitKey(100000)

       
        
        #cv2.imshow("Head Pose Estimation", image)
        
        
        #if headPoseDetected == True:
            
            
        if cv2.waitKey(1) & 0xFF == 27:
            break
    else:
        if filename == "frame3340.jpg": 
            print("photoX")
        
    
            
#cap.release()
cv2.destroyAllWindows()

In [ ]:
print(x)
print(y)
print(z)
print(old_path)

In [ ]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Holistic model
mp_holistic = mp.solutions.holistic

# Read the image/frame
#ExtremeHeadPose
frame = cv2.imread('C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\Frames\\frame20.jpg')  # Or capture from video stream

# Convert the image to RGB (MediaPipe holistic model works on RGB images)
rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
img_h, img_w, img_c = frame.shape
print(frame.shape)
# Initialize holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Process the frame
    results = holistic.process(rgb_frame)

    # Extract head pose angles
    if results.pose_landmarks:
        pose_landmarks = results.pose_landmarks.landmark
        # Extract head pose angles (yaw, pitch, roll)
        head_pose = (pose_landmarks[mp_holistic.PoseLandmark.NOSE].visibility, pose_landmarks[mp_holistic.PoseLandmark.NOSE].z, pose_landmarks[mp_holistic.PoseLandmark.NOSE].y)



    # Display the result
    x = pose_landmarks[mp_holistic.PoseLandmark.NOSE].x*img_w
    y = pose_landmarks[mp_holistic.PoseLandmark.NOSE].y*img_h
    p1 =(int(x), int(y))
    p2 =(int(x+100), int(y-100))
            
    cv2.line(frame,p1,p2,(255,0,0),3)
    print(p1)
    print(p2)
    print("Head Pose Angles :", head_pose)
    print(pose_landmarks[mp_holistic.PoseLandmark.NOSE].x)
    cv2.imshow('Head Pose Estimation', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()